# Dystopedia

I once read [a Tweet](https://twitter.com/lbcyber/status/1115015586243862528) that said,
“you can make any Wikipedia article dystopian by changing it to the past tense.”
Dystopian Wikipedia sounded <s>fun</s> fitting for our quasi-dystopian times,
so I built Dystopedia as a proof-of-concept.

Try the demo on [🤗 Spaces](https://huggingface.co/spaces/codesue/dystopedia),
and learn how I made it on [codesue.com](https://codesue.com/posts/dystopedia.).

Fetch the first paragraph (~7 sentences) of the Wikipedia article for "water".

In [ ]:
import wikipedia


result = wikipedia.search("water", results=1)
summary = wikipedia.summary(
  result,
  sentences=7,
  auto_suggest=False,
  redirect=True
)

print(summary)

Convert present tense verbs to past tense.

In [ ]:
import lemminflect
import spacy


nlp = spacy.load("en_core_web_lg")

def make_past_tense(token):
  if token.tag_ in ("VBP", "VBZ"):
    return f'{token._.inflect("VBD")} '
  return token.text_with_ws

doc = nlp(summary)
dystopian_summary = "".join(
  [make_past_tense(token) for token in doc]
)

print(dystopian_summary)

Create a function to determine whether a given text is positive.

In [ ]:
from transformers import pipeline


sentiment_analyzer = pipeline(
  "sentiment-analysis",
  model="distilbert-base-uncased-finetuned-sst-2-english",
  revision="af0f99b"
)

def is_positive(text):
  return sentiment_analyzer(text)[0]["label"] == "POSITIVE"

for word in ("good", "bad"):
  if is_positive(word):
    print(f'"{word}" was labelled positive')
  else:
    print(f'"{word}" was not labelled positive')

Classsify the summary for water.

In [ ]:
print(is_positive(summary))

Classify the first sentence of the summary.

In [ ]:
sentences = list(doc.sents)

print(sentences[0])
print(is_positive(sentences[0].text))

Classify the second sentence.

In [ ]:
print(sentences[1])
print(is_positive(sentences[1].text))

Classify the term "water" on its own.

In [ ]:
print(is_positive("water"))

Create a function that converts present tense verbs to the past tense when given a positive term.

In [ ]:
def make_dystopian(term, text):
  doc = nlp(text)
  if is_positive(term):
    return "".join([make_past_tense(token) for token in doc])
  return doc.text


for word in ("good", "bad"):
  print(make_dystopian(word, f"{word.capitalize()} things exist."))

Build a quck demo using Gradio.

In [ ]:
import gradio as gr


def get_dystopian_summary(term):
  if term == "":
    return term
  try:
    results = wikipedia.search(term, results=1)
  except wikipedia.exceptions.DisambiguationError as e:
    raise gr.Error(e.error)
  if len(results) == 0:
    raise gr.Error(
      f'Could not find an article on the term "{term}". '
      'Try searching for a different topic.'
    )
  summary = wikipedia.summary(results[0], sentences=1, auto_suggest=False, redirect=True)
  return make_dystopian(term, summary)

print(get_dystopian_summary("water"))

In [ ]:
def launch_demo(**kwargs):
  title = "Dystopedia"
  description = (
    "Make any Wikipedia topic dystopian. Inspired by [this Tweet](https://twitter.com/lbcyber/status/1115015586243862528). "
    "Dystopedia uses [DistilBERT base uncased finetuned SST-2](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) "
    "for sentiment analysis and is subject to its limitations and biases."
  )
  examples = ["joy", "hope", "peace", "Earth", "water", "food"]
  gr.Interface(
    fn=get_dystopian_summary,
    inputs=gr.Textbox(label="term", placeholder="Enter a term...", max_lines=1),
    outputs=gr.Textbox(label="description"),
    title=title,
    description=description,
    examples=examples,
    cache_examples=True,
    allow_flagging="never",
  ).launch(**kwargs)

# If running this notebook remotely on binder, use launch_demo(share=True)
launch_demo(show_error=True)